<a href="https://colab.research.google.com/github/gopalkalpande/fastai-huggingface-study-group/blob/my-playbook/fastai%2Bhuggingface_session_1_Transformers%2C_what_can_they_do%3F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformers, what can they do?

Install the Transformers and Datasets libraries to run this notebook.

In [1]:
!pip install -qq transformers[sentencepiece]
!pip install -qq datasets

from transformers import pipeline

     |████████████████████████████████| 2.5MB 30.3MB/s 
     |████████████████████████████████| 3.3MB 48.1MB/s 
     |████████████████████████████████| 901kB 41.1MB/s 
     |████████████████████████████████| 1.1MB 37.0MB/s 
     |████████████████████████████████| 266kB 35.5MB/s 
     |████████████████████████████████| 245kB 52.8MB/s 
     |████████████████████████████████| 122kB 50.3MB/s 


## Tokenizer basics

In [2]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')
print(type(tokenizer))
print(tokenizer)


<class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>
PreTrainedTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
print(type(tokenizer))
print(tokenizer)


<class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>
PreTrainedTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [4]:
inputs = tokenizer('hi my name is Wayde from ohmeow.com')

print(type(inputs), type(inputs.data))
print(inputs.data)

<class 'transformers.tokenization_utils_base.BatchEncoding'> <class 'dict'>
{'input_ids': [101, 20844, 1139, 1271, 1110, 4714, 2007, 1121, 9294, 3263, 4064, 119, 3254, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [6]:
len(inputs.data["input_ids"]), len(inputs.data["token_type_ids"]), len(inputs.data["attention_mask"])

(14, 14, 14)

In [13]:
for idx, tok in enumerate(tokenizer.vocab.keys()):
  print(f'{tok}: {tokenizer.vocab[tok]}')
  if (idx >4): break

loose: 5768
forestry: 21292
##iae: 21845
##ivo: 15435
Fairbanks: 27591
##el: 1883


In [15]:
print(tokenizer.decode(inputs.input_ids))
# print(tokenizer.decode(inputs.input_ids))
print(tokenizer.convert_ids_to_tokens(inputs.input_ids))
print(len(tokenizer.convert_ids_to_tokens(inputs.input_ids)))


[CLS] hi my name is Wayde from ohmeow. com [SEP]
['[CLS]', 'hi', 'my', 'name', 'is', 'Way', '##de', 'from', 'oh', '##me', '##ow', '.', 'com', '[SEP]']
14


## Sentiment

In [16]:
classifier = pipeline("sentiment-analysis")
classifier("I've been waiting for a HuggingFace course my whole life.")

[{'label': 'POSITIVE', 'score': 0.9598048329353333}]

In [17]:
classifier([
    "I've been waiting for a HuggingFace course my whole life.", 
    "I hate this so much!"
])

[{'label': 'POSITIVE', 'score': 0.9598048329353333},
 {'label': 'NEGATIVE', 'score': 0.9994558095932007}]

In [19]:
classifier.model.name_or_path, classifier.modelcard

('distilbert-base-uncased-finetuned-sst-2-english', None)

Sentiment using a user-contributed model [from the model hub](https://huggingface.co/oliverguhr/german-sentiment-bert?text=Das+it+gut).  Just click on the "Use in Transformers" button to grab the code you need to use it in your code.  There are also options for getting everything you need to run it using the accelerated API and/or in Sagemaker ... there is even the ability to use the inference API to run some tests before you download right on the model's page!

In [20]:
classifier = pipeline("sentiment-analysis", model='oliverguhr/german-sentiment-bert')
print(f'Results: {classifier("Das is gut!")}')
print(classifier.model.name_or_path, classifier.modelcard)


Results: [{'label': 'positive', 'score': 0.9705124497413635}]
oliverguhr/german-sentiment-bert None


## Zero-shot

In [22]:
classifier = pipeline("zero-shot-classification")
classifier(
    "This is a course about the Transformers library",
    candidate_labels=["education", "politics", "business", "coding", "dancing"],
)

{'labels': ['education', 'coding', 'business', 'dancing', 'politics'],
 'scores': [0.6652492880821228,
  0.165022075176239,
  0.08819849789142609,
  0.04732424020767212,
  0.03420596569776535],
 'sequence': 'This is a course about the Transformers library'}

In [23]:
classifier.model.name_or_path, classifier.modelcard

('facebook/bart-large-mnli', None)

## Text generation

In [27]:
generator = pipeline("text-generation")
generator(["In this course, we will teach you how to",
           "I'm feeling quite low, I think I'm bad"])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[[{'generated_text': 'In this course, we will teach you how to use the Java EE 5 EE5 MIME Authentication Provider Framework and Java EE 8 Mockups to authenticate your web app. We will also show you how to get started with MIME authentication.\n'}],
 [{'generated_text': "I'm feeling quite low, I think I'm bad. I'm feeling slightly more like myself. (laughs) I guess I know that's just how it looks for guys. We just talk so much in the dressing room, so we just have"}]]

In [25]:
generator.model.name_or_path, generator.modelcard


('gpt2', None)

You can control how many different sequences are randomly generated via `num_return_sequences` and the total output length via `max_length` arguments.

See this article for info about text generation parameters:
https://huggingface.co/blog/how-to-generate

In [29]:
generator = pipeline("text-generation", model="distilgpt2")
generator(
    ["In this course, we will teach you how to",
           "I'm feeling quite low, I think I'm bad"],
    max_length=30,
    num_return_sequences=2,
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[[{'generated_text': 'In this course, we will teach you how to create a simple and effective method with a small amount of ingredients including ingredients, spices and a pinch of'},
  {'generated_text': 'In this course, we will teach you how to use your favorite software without needing to do it yourself. Learn to create a secure solution by using secure'}],
 [{'generated_text': 'I\'m feeling quite low, I think I\'m bad so I\'m trying to improve a bit on this," said the 31-year-old,'},
  {'generated_text': 'I\'m feeling quite low, I think I\'m bad too."\n\nThat\'s what she wants to tell them. "That is what I want'}]]

## Language modeling - MLM

In [33]:
unmasker = pipeline("fill-mask")
unmasker("This course will <mask> you all about BERT models.", top_k=5)

[{'score': 0.8721699118614197,
  'sequence': 'This course will teach you all about BERT models.',
  'token': 6396,
  'token_str': ' teach'},
 {'score': 0.1027325838804245,
  'sequence': 'This course will tell you all about BERT models.',
  'token': 1137,
  'token_str': ' tell'},
 {'score': 0.00842359196394682,
  'sequence': 'This course will show you all about BERT models.',
  'token': 311,
  'token_str': ' show'},
 {'score': 0.006370928138494492,
  'sequence': 'This course will remind you all about BERT models.',
  'token': 8736,
  'token_str': ' remind'},
 {'score': 0.003180696628987789,
  'sequence': 'This course will inform you all about BERT models.',
  'token': 6296,
  'token_str': ' inform'}]

In [35]:
#@title why it masks only 1 mask? why it doesnot works with multiple masks
unmasker.model.name_or_path, unmasker.modelcard

('distilroberta-base', None)

## Token classification (e.g., NER)

What does `grouped_entities` do?

In [37]:
ner = pipeline("ner", grouped_entities=True)
ner(["My name is Sylvain and I work at Hugging Face in Brooklyn.",
     "The solution should be bolied at 100 degree celsius before flowing through the tubes.",
     "Gopal and Vikas went for dinner and they enjoyed it."])

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/token_classification.py:155: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  f'`grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="{aggregation_strategy}"` instead.'


[[{'end': 18,
   'entity_group': 'PER',
   'score': 0.9981694,
   'start': 11,
   'word': 'Sylvain'},
  {'end': 45,
   'entity_group': 'ORG',
   'score': 0.97960204,
   'start': 33,
   'word': 'Hugging Face'},
  {'end': 57,
   'entity_group': 'LOC',
   'score': 0.99321055,
   'start': 49,
   'word': 'Brooklyn'}],
 [],
 [{'end': 5,
   'entity_group': 'PER',
   'score': 0.9878795,
   'start': 0,
   'word': 'Gopal'},
  {'end': 15,
   'entity_group': 'PER',
   'score': 0.872009,
   'start': 10,
   'word': 'Vikas'}]]

In [40]:
ner = pipeline("ner")#, grouped_entities=False)
ner(["My name is Sylvain and I work at Hugging Face in Brooklyn.",
     "The solution should be bolied at 100 degree celsius before flowing through the tubes.",
     "Gopal and Vikas went for dinner and they enjoyed it."])

[[{'end': 12,
   'entity': 'I-PER',
   'index': 4,
   'score': 0.9993828,
   'start': 11,
   'word': 'S'},
  {'end': 14,
   'entity': 'I-PER',
   'index': 5,
   'score': 0.99815476,
   'start': 12,
   'word': '##yl'},
  {'end': 16,
   'entity': 'I-PER',
   'index': 6,
   'score': 0.99590725,
   'start': 14,
   'word': '##va'},
  {'end': 18,
   'entity': 'I-PER',
   'index': 7,
   'score': 0.9992327,
   'start': 16,
   'word': '##in'},
  {'end': 35,
   'entity': 'I-ORG',
   'index': 12,
   'score': 0.97389334,
   'start': 33,
   'word': 'Hu'},
  {'end': 40,
   'entity': 'I-ORG',
   'index': 13,
   'score': 0.976115,
   'start': 35,
   'word': '##gging'},
  {'end': 45,
   'entity': 'I-ORG',
   'index': 14,
   'score': 0.98879766,
   'start': 41,
   'word': 'Face'},
  {'end': 57,
   'entity': 'I-LOC',
   'index': 16,
   'score': 0.99321055,
   'start': 49,
   'word': 'Brooklyn'}],
 [],
 [{'end': 2,
   'entity': 'I-PER',
   'index': 1,
   'score': 0.9900885,
   'start': 0,
   'word': 'Go'}

## Question answering

In [41]:
question_answerer = pipeline("question-answering")
question_answerer(
    question="Where do I work?",
    context="My name is Sylvain and I work at Hugging Face in Brooklyn"
)

{'answer': 'Hugging Face', 'end': 45, 'score': 0.6949757933616638, 'start': 33}

## Summarization

In [42]:
summarizer = pipeline("summarization")
summarizer("""
    America has changed dramatically during recent years. Not only has the number of 
    graduates in traditional engineering disciplines such as mechanical, civil, 
    electrical, chemical, and aeronautical engineering declined, but in most of 
    the premier American universities engineering curricula now concentrate on 
    and encourage largely the study of engineering science. As a result, there 
    are declining offerings in engineering subjects dealing with infrastructure, 
    the environment, and related issues, and greater concentration on high 
    technology subjects, largely supporting increasingly complex scientific 
    developments. While the latter is important, it should not be at the expense 
    of more traditional engineering.

    Rapidly developing economies such as China and India, as well as other 
    industrial countries in Europe and Asia, continue to encourage and advance 
    the teaching of engineering. Both China and India, respectively, graduate 
    six and eight times as many traditional engineers as does the United States. 
    Other industrial countries at minimum maintain their output, while America 
    suffers an increasingly serious decline in the number of engineering graduates 
    and a lack of well-educated engineers.
""")

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


[{'summary_text': ' America has changed dramatically during recent years . The number of engineering graduates in the U.S. has declined in traditional engineering disciplines such as mechanical, civil,    electrical, chemical, and aeronautical engineering . Rapidly developing economies such as China and India continue to encourage and advance the teaching of engineering .'}]

## Translation

In [49]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-mr")
translator("Ce cours est produit par Hugging Face.")

[{'translation_text': '(g०१ ११ / ८)'}]

In [50]:
translator("I am Gopal Kalpande.")


[{'translation_text': 'मी आयरिश आहे.'}]

In [51]:
ls

sample_data/
